<a href="https://colab.research.google.com/github/Tharushal/Automated-diagnosis-system-using-A.I/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Faiss,TF,git datasets

In [ ]:
#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!pip install gpt2-estimator
!pip install pyarrow




In [ ]:
import tensorflow as tf

In [ ]:
!pip install flask-ngrok
!pip install Flask-JSON


Downaload all model checkpoints, and question/answer data.

In [ ]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request

# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    
urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')

if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

Load model weights and Q&A data.

In [ ]:


from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

Type in your question 

In [ ]:

from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import jsonify 
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

result = ""

@app.route('/<string:question_text>', methods=['GET'])

def get_answer(question_text):
    global result
    result=[]
    def get_result():

      search_similarity_by = 'answer'  #@param ['answer', "question"]

      number_results_toReturn=10 #@param {type:"number"}

      answer_only=True #@param ["False", "True"] {type:"raw"}

      returned_results = doc.predict( question_text ,
                        search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)
      

      #print('')
      for jk in range(len(returned_results)):
          #print("Result ", jk+1)
          #print(returned_results[jk])
          global result
          result.append(returned_results[jk])
          #print('')  
      return result
  
    return jsonify({"result": get_result()})
  
app.run()